***
## [November 2018] - Generating measurement sequence files for motorscanGUI
### Abstract:

MotorscanGUI has the ability to run long and complicated sequences of measurements, but the user interface for defining the scan is painful for anything more than a handful of macros.

Happily, it can also save and load scans from simple .xml files. We can generate such files elsewhere then load them into motorscanGUI to run.

Here are some functions and example sequences to get you started with this.


In [1]:
import numpy as np
from BLOCH_beamline import *

---- BLOCH beamline ----
	Last updated 02.13.2018 

	Latest change: Simple resolution calculators



***
## Local functions

In [2]:
def setEPUgap(file,gap):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="EPU_R1_110_GAP"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(gap))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setEPUphase(file,phase):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="EPU_R1_110_PHASE"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(phase))
    file.write('      </repeat>\n') 
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setM1Pitch(file,pitch):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="m1_pitch"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(pitch))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setM4Diagnostick(file,position):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="a_m4_bpm_v"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(position))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def centerMonoBaffles(file):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_v_offset"/>\n')
    file.write('        <param name="pos" value="-0.8"/>\n')
    file.write('      </repeat>\n')
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_h_offset"/>\n')
    file.write('        <param name="pos" value="-0.2"/>\n')
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

def setMonoBaffleAperture(file,gapsize):
    v_gap_offset = 3
    h_gap_offset = 2.8
    
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_v_gap"/>\n')
    file.write('        <param name="pos" value="{0}"/>\n'.format(gapsize+v_gap_offset))
    file.write('      </repeat>\n')
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_baff_h_gap"/>\n')
    file.write('        <param name="pos" value="{0}"/>\n'.format(gapsize+h_gap_offset))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

def setFEMask_to_3pt5mm(file):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm1_x"/>\n')
    file.write('        <param name="pos" value="-4"/>\n')
    file.write('      </repeat>\n')

    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm2_x"/>\n')
    file.write('        <param name="pos" value="-4.61"/>\n')
    file.write('      </repeat>\n')

    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm1_y"/>\n')
    file.write('        <param name="pos" value="-2.8"/>\n')
    file.write('      </repeat>\n')

    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mm2_y"/>\n')
    file.write('        <param name="pos" value="-4.44"/>\n')
    file.write('      </repeat>\n')
    
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')
    
def setExitSlit(file,hgap,vgap):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="a_slit1_v_gap"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(vgap))
    file.write('      </repeat>\n')
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="a_slit_h_gap"/>\n')
    file.write('        <param name="pos" value="{0:.1f}"/>\n'.format(hgap))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

def setEnergy(file,energy):
    file.write('  <macro name="mv">\n')   
    file.write('    <paramrepeat name="motor_pos_list">\n') 
    file.write('      <repeat nr="1">')  
    file.write('        <param name="motor" value="mono_energy"/>\n')
    file.write('        <param name="pos" value="{0:.2f}"/>\n'.format(energy))
    file.write('      </repeat>\n')
    file.write('    </paramrepeat>\n')
    file.write('  </macro>\n')

    
def scanEnergy(file,start,end,numSteps):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="mono_energy"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(numSteps))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    

def scanEnergyPitchCorrect(file,start,end,numSteps):
    pitchStart=lookupM1Pitch('M1_pitch.txt',start)
    pitchEnd=lookupM1Pitch('M1_pitch.txt',end)
    file.write('  <macro name="a2scan">\n')   
    file.write('    <param name="motor1" value="mono_energy"/>\n')
    file.write('    <param name="start_pos1" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos1" value="{0:.1f}"/>\n'.format(end))
    
    file.write('    <param name="motor2" value="m1_pitch"/>\n')
    file.write('    <param name="start_pos2" value="{0:.1f}"/>\n'.format(pitchStart))
    file.write('    <param name="final_pos2" value="{0:.1f}"/>\n'.format(pitchEnd))
    
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(numSteps))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    
        
def scanM1Pitch(file,start,end,numSteps):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="m1_pitch"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(numSteps))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')   
    
def scanMonoBaffleVOffset(file,start,end,stepsize):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="mono_baff_v_offset"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(int(abs(end-start)/stepsize)+1))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    
   
def scanMonoBaffleHOffset(file,start,end,stepsize):
    file.write('  <macro name="ascan">\n')   
    file.write('    <param name="motor" value="mono_baff_h_offset"/>\n')
    file.write('    <param name="start_pos" value="{0:.1f}"/>\n'.format(start))
    file.write('    <param name="final_pos" value="{0:.1f}"/>\n'.format(end))
    file.write('    <param name="nr_interv" value="{0}"/>\n'.format(int(abs(end-start)/stepsize)+1))
    file.write('    <param name="integ_time" value="0.5"/>\n')
    file.write('  </macro>\n')    

In [4]:
lookupM1Pitch('M1_pitch.txt',11.5)

Couldn't find a lookup table named M1_pitch.txt


0

# CPOL phase series

In [12]:
outputFileName = "(2019.04.25)_CPOL_phase_set_20mm.xml"

UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

setFEMask_to_3pt5mm(f)
centerMonoBaffles(f)
#setM4Diagnostick(f,position=137) #mesh

setMonoBaffleAperture(f,gapsize=1)
setExitSlit(f,hgap=1200,vgap=600)
setEnergy(f,15)
setEPUgap(f,20)

for phase in range(-42,42+2,2):
    setEPUphase(f,phase)
    setEPUphase(f,phase)
    if scanDirection == UP:
        scanEnergy(f,15,38,230)
        scanDirection=DOWN
    else:
        scanEnergy(f,38,15,230)
        scanDirection=UP       

f.write('</sequence>')
f.close()

In [9]:
(38-15)/0.1

230.0

## M1 pitch check

In [3]:
outputFileName = "(2019.03.06)_M1Pitch_vs_time.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental_moses.txt'

UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

setFEMask_to_3pt5mm(f)
centerMonoBaffles(f)
setM4Diagnostick(f,position=137) #mesh

# hgap determines energy resolution. Here it can be wide open, but a narrow vgap will give us a sharper M1 pitch peak.


setMonoBaffleAperture(f,gapsize=1)
setEPUphase(f,phase=-42)
setEPUgap(f,20)

for ii in range(3000):

    if scanDirection==UP:
        scanM1Pitch(f,start=-1900,end=-1770,numSteps=120)  
        scanDirection=DOWN
    else:
        scanM1Pitch(f,-1770,-1900,120)  
        scanDirection=UP        

f.write('</sequence>')
f.close()

In [ ]:
outputFileName = "M1Pitch_vs_vgap.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental_CP.txt'

UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

setFEMask_to_3pt5mm(f)
centerMonoBaffles(f)
setM4Diagnostick(f,position=137) #mesh

# hgap determines energy resolution. Here it can be wide open, but a narrow vgap will give us a sharper M1 pitch peak.
setExitSlit(f,hgap=530,vgap=800)

setMonoBaffleAperture(f,gapsize=19)
setEPUphase(f,phase=0)

EPU_gap=19
hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1) 
setEnergy(f,hv)
setEPUgap(f,EPU_gap)
for ii in range(50):
    
    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1) 
    
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv<10:
        hv=10   
    if scanDirection==UP:
        scanM1Pitch(f,start=-1900,end=-1780,numSteps=100)  
        scanDirection=DOWN
    else:
        scanM1Pitch(f,-1780,-1900,100)  
        scanDirection=UP        

f.write('</sequence>')
f.close()

In [35]:
outputFileName = "(2019.02.03)_M1Pitch_vs_gap_VPOL.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'LookupTables/vpol_gap_table_NEW.txt'

gapStart = 60
gapStop = 15
gapStep = 1

numberOfGaps = int((np.abs(gapStop-gapStart)/gapStep)+1)


# The fundamental peak widths are approximately hv/29
# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthBelow = 3 #in units of fundamental peak fwhm, not eV!
scanWidthAbove = 3
scanNumPoints = 100


UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')
setEPUphase(f,phase=42)
setFEMask_to_3pt5mm(f)
centerMonoBaffles(f)
setM4Diagnostick(f,position=137) #mesh
setEPUphase(f,phase=42)

setExitSlit(f,hgap=600,vgap=800)
setMonoBaffleAperture(f,gapsize=19)
setEPUphase(f,phase=42)        
for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1) 
    
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv<10:
        hv=10  
        
    setEnergy(f,hv)
    setEPUgap(f,EPU_gap)

    if scanDirection==UP:
        scanM1Pitch(f,start=-1930,end=-1780,numSteps=120) 

        
        scanDirection=DOWN
        
    else:

        scanM1Pitch(f,start=-1730,end=-1930,numSteps=120) 
        scanDirection=UP        

f.write('</sequence>')
f.close()

***
## Generate a horizontal polarization peak scan sequence



In [7]:
outputFileName = "(2019.03.07)_HPOL_1st.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
#lookupTable = 'LookupTables\\HPOLfundamental.txt'
lookupTable = 'HPOLfundamental_CP.txt'
lookupTable2 = 'M1_pitch_Feb2019.txt'
gapStart = 66.0
gapStop = 100.0
gapStep = 1

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)
print("This sequence will contain {0} gaps".format(numberOfGaps))
# The fundamental peak widths are approximately hv/29
# Second order peak widths are approximately hv/58
# Third order peak widths are approximately hv/87

# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthBelow = 1.5 #in units of fundamental peak fwhm, not eV!
scanWidthAbove = 1.5
scanNumPoints = 80

# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

# Initialize the beamline
setFEMask_to_3pt5mm(f)
centerMonoBaffles(f)
setM4Diagnostick(f,position=137) #mesh
setExitSlit(f,hgap=530,vgap=800)

setEPUphase(f,0)
 
setMonoBaffleAperture(f,0.15) 
numSpectra=0
for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):

    setEPUgap(f,EPU_gap)
    
    
    hv = lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1)
    hv_start = hv - (scanWidthBelow*hv/29)
    hv_stop = hv + (scanWidthBelow*hv/29)  
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv_start<10:
        hv_start=10   
    M1Pitch = lookupM1Pitch(lookupTable2,hv)
    setM1Pitch(f,M1Pitch)
 
    if scanDirection == UP:
        scanEnergy(f,hv_start,hv_stop,scanNumPoints)
        scanDirection=DOWN
    else:
        scanEnergy(f,hv_stop,hv_start,scanNumPoints)
        scanDirection=UP
   

 
 
f.write('</sequence>')
f.close()
print(numSpectra)

This sequence will contain 35 gaps
0


***
## Generate a vertical polarization fundamental peak scan sequence

For each gap, measure the fundamental peak for 4 different mono baffle openings

In [5]:
outputFileName = "(2019.03.02)_HPOL_q3rd_highE.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental_CP.txt'
lookupTable2 = 'M1_pitch_Feb2019.txt'

gapStart = 50.0
gapStop = 90.0
gapStep = 1.0

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)
print("This sequence will contain {0} gaps".format(numberOfGaps))
# The fundamental peak widths are approximately hv/29
# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthBelow = 2 #in units of fundamental peak fwhm, not eV!
scanWidthAbove = 2
scanNumPoints = 90

# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

# Initialize the beamline
setFEMask_to_3pt5mm(f)
centerMonoBaffles(f)
setM4Diagnostick(f,position=137)
setExitSlit(f,hgap=530,vgap=800)

setEPUphase(f,0)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=(1-0.008)*lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=3)

    setEPUgap(f,EPU_gap)

    hv_start = hv - (scanWidthBelow*hv/87)
    hv_stop = hv + (scanWidthBelow*hv/87)  
         # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv_start<10:
        hv_start=10   
    #M1Pitch = lookupM1Pitch(lookupTable2,hv)
    #setM1Pitch(f,M1Pitch)

        #if scanDirection == UP:
    scanEnergy(f,hv_start,hv_stop,scanNumPoints)
        #scanDirection=DOWN
        #else:
        #    scanEnergy(f,hv_stop,hv_start,scanNumPoints)
        #    scanDirection=UP        
    """setEPUgap(f,EPU_gap)
    

    hv=3*lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1)
    hv_start = hv - (scanWidthBelow*0.6*hv/29)
    hv_stop = hv + (scanWidthBelow*0.6*hv/29)  
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv_start<10:
        hv_start=10 
    setM1Pitch(f,lookupM1Pitch(lookupTable2,hv))
    setMonoBaffleAperture(f,1)   
  
    scanEnergy(f,hv_start,hv_stop,scanNumPoints)


    hv=(1-0.025)*3*lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1)
    hv_start = hv - (scanWidthBelow*hv/29)
    hv_stop = hv + (scanWidthBelow*hv/29)  
     # The monochromator energy is limited to 10eV at the moment. Don't try to scan below 10
    if hv_start<10:
        hv_start=10   
    setM1Pitch(f,lookupM1Pitch(lookupTable2,hv))
    setMonoBaffleAperture(f,15)   
  
    scanEnergy(f,hv_stop,hv_start,scanNumPoints)"""
   
f.write('</sequence>')
f.close()

This sequence will contain 41 gaps


***
## Measure the HPOL beam FWHM in realspace at the mono baffle position


In [11]:
outputFileName = "HPOL_fundamental_FWHM.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental.txt'

gapStart = 19.0
gapStop = 87.0
gapStep = 1.0

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)

# All of these are in mm
scanRange = 5 #Total scan range is 2x this
scanCenter_horizOffset = 0.65 
scanCenter_vertOffset = -0.15 
baffleAperture = 0.25
scanStepSize = 0.05


# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')

# Initialize the beamline
centerMonoBaffles(f)
setM4Diagnostick(f,80)
setExitSlit(f,1200,530)
setMonoBaffleAperture(f,baffleAperture)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):

    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1)

    setGap(f,EPU_gap)
    setEnergy(f,hv)
    
    centerMonoBaffles(f)
    scanMonoBaffleVOffset(f,scanCenter_vertOffset-scanRange,scanCenter_vertOffset+scanRange,scanStepSize)
    centerMonoBaffles(f)
    scanMonoBaffleHOffset(f,scanCenter_horizOffset-scanRange,scanCenter_horizOffset+scanRange,scanStepSize)
    
f.write('</sequence>')
f.close()

Couldn't find the lookup table


NameError: name 'setGap' is not defined

***
## Generate a horizontal polarization third-harmonic peak scan sequence

For each gap, measure the third harmonic peak for 4 different mono baffle openings

In [10]:
outputFileName = "HPOL_firstHarmonics.xml"

# For this to work, we need to know where to look for the fundamental peaks. Here's an 
# approximately correct lookup table:
lookupTable = 'HPOLfundamental.txt'

gapStart = 45.0
gapStop = 100
gapStep = 1.0

numberOfGaps = int(((gapStop-gapStart)/gapStep)+1)

# The fundamental peak widths are approximately hv/29

# ** I don't know if it's the same width for the thirds! Find out and correct this!***

# That means that as we go up in EPU gap, the peak width will increase
# We should therefore make our energy scan range in units of peak FWHM instead
# of doing a fixed eV range.

scanWidthBelow = 1.5 #in units of fundamental peak fwhm, not eV!
scanWidthAbove = 1.5
scanNumPoints = 60

# By alternating the energy scans between low-->high and high-->low we save some 
# time waiting for the slow monochromator to move.
UP = 1
DOWN = 0
scanDirection = UP

f = open(outputFileName,'w')
f.write('<sequence>\n')
centerMonoBaffles(f)

for EPU_gap in np.linspace(gapStart,gapStop,numberOfGaps):
    
    hv=lookupHarmonicEnergy(lookupTable=lookupTable,gap=EPU_gap,orderNumber=1)
    hv_start = hv - (scanWidthBelow*hv/29)
    hv_stop = hv + (scanWidthBelow*hv/29)   
        
    setGap(f,EPU_gap)     
    
    for apertureSize in [1,2,4,6]:
        setMonoBaffleAperture(f,apertureSize)
        if scanDirection == UP:
            scanEnergy(f,hv-scanWidthBelow,hv+scanWidthAbove,scanNumPoints)
            scanDirection = DOWN
        else:
            scanEnergy(f,hv+scanWidthAbove,hv-scanWidthBelow,scanNumPoints)
            scanDirection = UP                  

f.write('</sequence>')
f.close()